In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os, ast, re
import cv2

In [2]:
classify_interpreter = tf.lite.Interpreter(model_path="converted_currency_mobilenetmodel.tflite")
watermark_interpreter = tf.lite.Interpreter(model_path="converted_watermark_mobilenetmodel.tflite")
uv_interpreter = tf.lite.Interpreter(model_path="converted_uv_mobilenetmodel.tflite")

In [3]:
classify_input_details = classify_interpreter.get_input_details()
classify_output_details = classify_interpreter.get_output_details()
watermark_input_details = watermark_interpreter.get_input_details()
watermark_output_details = watermark_interpreter.get_output_details()
uv_input_details = uv_interpreter.get_input_details()
uv_output_details = uv_interpreter.get_output_details()

classify_interpreter.allocate_tensors()
watermark_interpreter.allocate_tensors()
uv_interpreter.allocate_tensors()

In [4]:
def get_labels(path):	
	f = open(path, "r")
	labels = f.read()
	labels = ast.literal_eval(labels)

	final_labels = {v: k for k, v in labels.items()}
	
	return final_labels
currency_labels = get_labels('mobilenet_currency_class_indices.txt')
uv_labels = get_labels('mobilenet_uv_class_indices.txt')
watermark_labels = get_labels('mobilenet_watermark_class_indices.txt')

In [5]:
def predict_image(imgname, final_labels, interpreter, input_details, output_details):
    test_image = cv2.imread(imgname)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    if final_labels==uv_labels:
        h, w, c = test_image.shape #height*width*channel - ex: 480,640,3
        x_start_r = 0.3125  
        x_end_r = 0.703125  
        y_start_r = 0.270833
        y_end_r = 0.791666  
    
        #np.floor-round of lowest integer value
        x = int(np.floor(x_start_r * w))#200
        y = int(np.floor(y_start_r * h))# 130
        x2 = int(np.floor(x_end_r * w))#450
        y2 = int(np.floor(y_end_r * h))# 380

        test_image = test_image[y:y2, x:x2].copy() #copying image to image parameter
        #[130:380,200:450]-croping image for considering security thread
    else:
        test_image = cv2.resize(test_image, (224, 224), cv2.INTER_AREA)
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    test_image = (2.0 / 255.0) * test_image - 1.0
    test_image = test_image.astype(np.float32)

    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]['index'])

    result_dict = dict()
    for key in list(final_labels.keys()):
        result_dict[final_labels[key]] = result[0][key]
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1], reverse=True)}
    final_result = dict()
    final_result[list(sorted_results.keys())[0]] = sorted_results[list(sorted_results.keys())[0]] * 100

    return final_result

In [6]:
path_classify=input()
res_classify=predict_image(path_classify, currency_labels, classify_interpreter, classify_input_details, classify_output_details)
if res_classify.get("invalid") is not None:
    print(res_classify)
    a=cv2.imread(path_classify)
    cv2.imshow('fake note',a)
    cv2.waitKey(0)
    print("Fake")
    cv2.destroyAllWindows()
else:
    print(res_classify)
    a=cv2.imread(path_classify)
    cv2.imshow('real note',a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("Real")

D:\Bec\Projects\BEC_Project\4-2\Project\CNN-based-classification-(PRIMARY)\batch-test-images\10back.jpg
{'10_new_back_up': 34.22751426696777}
Real


In [7]:
path_classify=input()
res_classify=predict_image(path_classify, currency_labels, classify_interpreter, classify_input_details, classify_output_details)
if res_classify.get("invalid") is not None:
    print(res_classify)
    a=cv2.imread(path_classify)
    cv2.imshow('fake note',a)
    cv2.waitKey(0)
    print("Fake")
    cv2.destroyAllWindows()
else:
    print(res_classify)
    a=cv2.imread(path_classify)
    cv2.imshow('real note',a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("Real")

D:\Bec\Projects\BEC_Project\4-2\Project\CNN-based-classification-(PRIMARY)\test-images\OT-2-11.jpg
{'invalid': 11.735772341489792}
Fake


In [8]:
path_uv=input()
res_uv=predict_image(path_uv, uv_labels, uv_interpreter, uv_input_details, uv_output_details)
if res_uv.get("nopatch") is not None:
    print(res_uv)
    a=cv2.imread(path_uv)
    cv2.imshow('fake note',a)
    cv2.waitKey(0)
    print("Fake")
    cv2.destroyAllWindows()
else:
    print(res_uv)
    a=cv2.imread(path_uv)
    cv2.imshow('real note',a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("Real")

D:\Bec\Projects\BEC_Project\4-2\Project\CNN-based-UV-validation-(PRIMARY)\test-images\fake.jpg
{'nopatch': 90.15783071517944}
Fake


In [9]:
path_wm=input()
res_wm=predict_image(path_wm, watermark_labels, watermark_interpreter, watermark_input_details, watermark_output_details)
if res_wm.get("no_watermark") is not None:
    print(res_wm)
    a=cv2.imread(path_wm)
    cv2.imshow('fake note',a)
    cv2.waitKey(0)
    print("Fake")
    cv2.destroyAllWindows()
else:
    print(res_wm)
    a=cv2.imread(path_wm)
    cv2.imshow('real note',a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("Real")

D:\Bec\Projects\BEC_Project\4-2\Project\CNN-based-watermark-validation-(PRIMARY)\test-images\yesww.jpg
{'yes_watermark': 99.86935257911682}
Real


In [3]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os, ast, re
import cv2

def get_labels(path):	
	f = open(path, "r")
	labels = f.read()
	labels = ast.literal_eval(labels)

	final_labels = {v: k for k, v in labels.items()}
	
	return final_labels

def predict_image(imgname, final_labels, interpreter, input_details, output_details):
    test_image = cv2.imread(imgname)
    test_image = cv2.resize(test_image, (224, 224), cv2.INTER_AREA)
    cv2.imshow('kappa', test_image)
    cv2.waitKey(0)
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)

    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]['index'])

    result_dict = dict()
    for key in list(final_labels.keys()):
        result_dict[final_labels[key]] = result[0][key]
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1], reverse=True)}

    for label in sorted_results.keys():
        print("{}: {}%".format(label, sorted_results[label] * 100))

    final_result = dict()
    final_result[list(sorted_results.keys())[0]] = sorted_results[list(sorted_results.keys())[0]] * 100

    return final_result


classify_interpreter = tf.lite.Interpreter(model_path="converted_currency_mobilenetmodel.tflite")
watermark_interpreter = tf.lite.Interpreter(model_path="converted_watermark_mobilenetmodel.tflite")
uv_interpreter = tf.lite.Interpreter(model_path="converted_uv_mobilenetmodel.tflite")

classify_input_details = classify_interpreter.get_input_details()
classify_output_details = classify_interpreter.get_output_details()
watermark_input_details = watermark_interpreter.get_input_details()
watermark_output_details = watermark_interpreter.get_output_details()
uv_input_details = uv_interpreter.get_input_details()
uv_output_details = uv_interpreter.get_output_details()

classify_interpreter.allocate_tensors()
watermark_interpreter.allocate_tensors()
uv_interpreter.allocate_tensors()

currency_labels = get_labels('mobilenet_currency_class_indices.txt')
uv_labels = get_labels('mobilenet_uv_class_indices.txt')
watermark_labels = get_labels('mobilenet_watermark_class_indices.txt')

res = predict_image(r'D:\Bec\Projects\BEC_Project\4-2\Project\CNN-based-classification-(PRIMARY)\test-images\2.jpg', currency_labels, classify_interpreter, classify_input_details, classify_output_details)
print("=" * 50)
print(res)



500_new_back_down: 16.78403466939926%
invalid: 8.921962976455688%
100_new_back_down: 6.527264416217804%
20_new_back_down: 5.860710516571999%
20_new_back_up: 5.613524466753006%
200_new_back_down: 5.180063843727112%
20_old_front_up: 4.490919038653374%
2000_new_back_up: 3.7207994610071182%
10_old_front_up: 2.977895736694336%
500_new_front_down: 2.5883732363581657%
50_old_back_up: 2.3997852578759193%
50_new_back_down: 2.2311674430966377%
10_old_back_down: 2.002578414976597%
200_new_front_up: 1.6380209475755692%
10_old_back_up: 1.6289569437503815%
500_new_back_up: 1.6174331307411194%
50_new_front_down: 1.505236979573965%
50_old_front_up: 1.4186342246830463%
200_new_back_up: 1.3539457693696022%
100_old_front_down: 1.289104949682951%
20_old_front_down: 1.2757597491145134%
20_old_back_up: 1.241471990942955%
50_old_front_down: 1.2031812220811844%
50_new_front_up: 1.1993111111223698%
50_old_back_down: 1.1987831443548203%
100_new_front_down: 1.1152454651892185%
50_new_back_up: 1.0489551350474358%